In [1]:
import pandas as pd
import urllib.request, json
import logging

In [3]:
# set up logging to monitor errors
logging.basicConfig(filename='../data/log/oac_geo.log', filemode='w', level=logging.DEBUG, format='%(asctime)s %(message)s')

In [3]:
# read in the excel file and specify sheet name
# change this to the new excel file name and sheet
df = pd.read_excel('../data/input/FY24 Older Adult Centers (11-20-23).xlsx', sheet_name='SC List by CMs 8a')

In [14]:
# create 'geoclient' column that will be sent to the api
# see: https://api.nyc.gov/geoclient/v1/doc/#section-1.3
df['geoclient'] = (df['Program Address'] + ", " + df['Borough'] + ", " + df['Zip Code'].astype(str))

In [16]:
# function to get the json data from the api
def json_get(url, row):
    try: 
        hdr ={
        # Request headers
        'Cache-Control': 'no-cache',
        # if you sign up for an account (https://api-portal.nyc.gov/) you can get a new subscription key and replace that here:
        'Ocp-Apim-Subscription-Key': '2fc22446e7af4d4eaa50d383ad846872',
        }

        req = urllib.request.Request(url, headers=hdr)

        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        logging.info(f'{row.Index}: {response.getcode()}')
        json_data = json.loads(response.read())
    except Exception as e:
        logging.info(e)
        json_data = {}
    return json_data

In [23]:
# function to get the lat/lon from the json data
def get_lat_lon(df):
    lat = []
    lon = []
    for row in df.itertuples():
        type = "search"
        url = f'https://api.nyc.gov/geo/geoclient/v1/search.json?input={row.geoclient}'
        json_data = json_get(url, row)

        try:
            lat.append(json_data['results'][0]['response']['latitude'])
            lon.append(json_data['results'][0]['response']['longitude'])
        except Exception as e:
            #logging.info(e.args)
            try:
                message = json_data['status']
                logging.info(f'{row.Index}: {message}')
            except Exception as e:
                logging.info(f'{row.Index}: {e}')
            lat.append('NA')
            lon.append('NA')
    return lat, lon

In [36]:
# replace spaces and new line with URL encoding for spaces
df['geoclient'] = df['geoclient'].replace(' ', '%20', regex=True)
df['geoclient'] = df['geoclient'].replace('\n', '%20', regex=True)

In [37]:
# run the function to get the lat/lon from geoclient api
df['lat'], df['lon'] = get_lat_lon(df)

In [38]:
# save the csv to data/output folder
df.to_csv('../data/output/df_geo.csv', index=False)